# RNN TIMIT demo

This notebook is an extension of the previous one on MLP. To find more details, read that one first.

*This is still work in progress!*

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES']='0'
#os.environ['KERAS_BACKEND']='tensorflow'

In [2]:
import numpy as np
from random import shuffle
from keras.models import Sequential
from keras.layers.core import *
from keras.layers.recurrent import *
from keras.layers.wrappers import *
from keras.optimizers import *
from IPython.display import clear_output
import time
import sys
from tqdm import *

sys.path.append('../python')

from data import Corpus, History

Using Theano backend.
Using gpu device 0: Tesla K40c (CNMeM is disabled, cuDNN 5004)

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
train=Corpus('../data/TIMIT_train.hdf5',load_normalized=True, merge_utts=True)
dev=Corpus('../data/TIMIT_dev.hdf5',load_normalized=True, merge_utts=True)
test=Corpus('../data/TIMIT_test.hdf5',load_normalized=True, merge_utts=True)

In [4]:
maxlen=max([train.getMaxLen(),dev.getMaxLen(),test.getMaxLen()])
print maxlen

777


In [5]:
import h5py

tr_in,tr_out_dec=train.get()
dev_in,dev_out_dec=dev.get()
tst_in,tst_out_dec=test.get()

h5_out = h5py.File('timit.h5', 'w')
dset = h5_out.create_dataset('train_x', tr_in.shape, dtype='f')
dset[...] = tr_in
dset = h5_out.create_dataset('dev_x', dev_in.shape, dtype='f')
dset[...] = dev_in
dset = h5_out.create_dataset('test_x', tst_in.shape, dtype='f')
dset[...] = tst_in
dset = h5_out.create_dataset('train_y', tr_out_dec.shape, dtype='i')
dset[...] = tr_out_dec
dset = h5_out.create_dataset('dev_y', dev_out_dec.shape, dtype='i')
dset[...] = dev_out_dec
dset = h5_out.create_dataset('test_y', tst_out_dec.shape, dtype='i')
dset[...] = tst_out_dec
h5_out.close()


print(tr_in.shape)
print(tr_out_dec.shape)
print(np.max(tr_out_dec))
for u in range(tr_in.shape[0]):
    tr_in[u]=tr_in[u][:,:26]
for u in range(dev_in.shape[0]):    
    dev_in[u]=dev_in[u][:,:26]
for u in range(tst_in.shape[0]):
    tst_in[u]=tst_in[u][:,:26]

(1124823, 39)
(1124823,)
60


IndexError: too many indices for array

In [ ]:
input_dim=tr_in[0].shape[1]
output_dim=61
hidden_num=140
epoch_num=50

def dec2onehot(dec):
    ret=[]
    for u in dec:
        assert np.all(u<output_dim)
        num=u.shape[0]
        r=np.zeros((num,output_dim))
        r[range(0,num),u]=1
        ret.append(r)
    return np.array(ret)

tr_out=dec2onehot(tr_out_dec)
dev_out=dec2onehot(dev_out_dec)
tst_out=dec2onehot(tst_out_dec)

## Model definition



In [ ]:
model = Sequential()

#model.add(LSTM_w_peepholes(input_dim=input_dim,output_dim=hidden_num,return_sequences=True))
model.add(LSTM(input_dim=input_dim,output_dim=hidden_num,return_sequences=True))
model.add(TimeDistributed(Dense(output_dim=output_dim)))
model.add(Activation('softmax'))

#optimizer = SGD(lr=4e-3, momentum=0.9, nesterov=False)
#optimizer= Adam(lr=0.0001)
optimizer=RMSprop(lr=4e-3)
loss='categorical_crossentropy'
metrics=['accuracy']
          
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
tr_hist=History('Train')
dev_hist=History('Dev')
tst_hist=History('Test')

tr_it=range(tr_in.shape[0])

for e in range(epoch_num):
    
    print 'Epoch #{}/{}'.format(e+1,epoch_num)
    sys.stdout.flush()
    
    shuffle(tr_it)
    for u in tqdm(tr_it):
        l,a=model.train_on_batch(np.array([tr_in[u]]),np.array([tr_out[u]]))
        tr_hist.r.addLA(l,a,tr_out[u].shape[0])        
    clear_output()    
    tr_hist.log()
    
    for u in range(dev_in.shape[0]):
        l,a=model.test_on_batch(np.array([dev_in[u]]),np.array([dev_out[u]]))
        dev_hist.r.addLA(l,a,dev_out[u].shape[0])
    dev_hist.log()
    
    
    for u in range(tst_in.shape[0]):
        l,a=model.test_on_batch(np.array([tst_in[u]]),np.array([tst_out[u]]))
        tst_hist.r.addLA(l,a,tst_out[u].shape[0])
    tst_hist.log()            
    
print 'Done!'
    

In [ ]:
import matplotlib.pyplot as P
%matplotlib notebook

fig,ax=P.subplots(2,sharex=True,figsize=(12,10))

ax[0].set_title('Loss')
ax[0].plot(tr_hist.loss,label='Train')
ax[0].plot(dev_hist.loss,label='Dev')
ax[0].plot(tst_hist.loss,label='Test')
ax[0].legend()

ax[1].set_title('PER %')
ax[1].plot(100*(1-np.array(tr_hist.acc)),label='Train')
ax[1].plot(100*(1-np.array(dev_hist.acc)),label='Dev')
ax[1].plot(100*(1-np.array(tst_hist.acc)),label='Test')
ax[1].legend()

In [ ]:
print 'Min test PER: {:%}'.format(1-np.max(tst_hist.acc))
print 'Min dev PER epoch: #{}'.format((np.argmax(dev_hist.acc)+1))
print 'Test PER on min dev: {:%}'.format(1-tst_hist.acc[np.argmax(dev_hist.acc)])